In [23]:
%pylab notebook
import read_mist_models # must have `read_mist_models.py` downloaded and in the same location as this notebook
from uncertainties import ufloat
from uncertainties.umath import * 
import pandas as pd
from scipy.interpolate import interp1d
#remove hashtags from the 2 lines below if not using magic
# import numpy as np
# import matplotlib.pyplot as plt

In [2]:
#####EDITABLE#####
#Insert name of binary system below
binary_system = '' 

#From the Gaia archive retrieve the parallax along with its uncertainty (website has it in units of milliarcseconds, mas)
parallax = None
error_parallax = None

#From the Astrophysical Parameters tab retrieve the magnitude of extinction A_G along with the lower and upper uncertainties- 
#-and the colour excess E(BP-RP) along with the lower and upper uncertainties 
A_G = None
lower_A_G = None
upper_A_G = None

E_BminusR = None
lower_E_BminusR = None
upper_E_BminusR = None

#Please check the very end of the param.out.8 files for each colour band to retrieve the best fit and one sigma values: 
#sfact (this is the light scale factor), LB/LA (this is the flux ratio)
scale_factor_G = None
error_scale_factor_G = None
flux_ratio_G = None
error_flux_ratio_G = None

scale_factor_BP = None
error_scale_factor_BP = None
flux_ratio_BP = None
error_flux_ratio_BP = None

scale_factor_RP = None
error_scale_factor_RP = None
flux_ratio_RP = None
error_flux_ratio_RP = None

In [3]:
#Creating ufloat variables to incorporate uncertainties in further calculations
para = ufloat(parallax,error_parallax)

error_A_G = (lower_A_G + upper_A_G)/2 #mean uncertainty in A_G
extinct_mag = ufloat(A_G,error_A_G)

error_E_BminusR = (lower_E_BminusR + upper_E_BminusR)/2 #mean uncertainty in E(BP-RP)
excess_col = ufloat(E_BminusR,error_E_BminusR)

sf_G = ufloat(scale_factor_G,error_scale_factor_G)
sf_BP = ufloat(scale_factor_BP,error_scale_factor_BP)
sf_RP = ufloat(scale_factor_RP,error_scale_factor_RP)

fr_G = ufloat(flux_ratio_G,error_flux_ratio_G)
fr_BP = ufloat(flux_ratio_BP,error_flux_ratio_BP)
fr_RP = ufloat(flux_ratio_RP,error_flux_ratio_RP)

#Converting scale factor values into flux
scale_factor_Ginflux = 10**(sf_G/-2.5)
scale_factor_BPinflux = 10**(sf_BP/-2.5)
scale_factor_RPinflux = 10**(sf_RP/-2.5)

#Calculating primary and secondary contributions in each colour band
primary_cont_G = 1/(fr_G+1)
secondary_cont_G = fr_G/(fr_G+1)
primary_cont_BP = 1/(fr_BP+1)
secondary_cont_BP = fr_BP/(fr_BP+1)
primary_cont_RP = 1/(fr_RP+1)
secondary_cont_RP = fr_RP/(fr_RP+1)

#Calculating apparent magnitude of each star in each colour band
mag_hot_G = -2.5 * log10(scale_factor_Ginflux*primary_cont_G)
mag_hot_BP = -2.5 * log10(scale_factor_BPinflux*primary_cont_BP)
mag_hot_RP = -2.5 * log10(scale_factor_RPinflux*primary_cont_RP)

mag_faint_G = -2.5 * log10(scale_factor_Ginflux*secondary_cont_G)
mag_faint_BP = -2.5 * log10(scale_factor_BPinflux*secondary_cont_BP)
mag_faint_RP = -2.5 * log10(scale_factor_RPinflux*secondary_cont_RP)

#Converting magnitude in G band to absolute magnitude and correcting it of extinction effects for plotting on y-axis
d_parsecs = 1/(para*(10**-3)) #converting from mas to parsecs
abs_mag_hot_G = mag_hot_G - 5*(log10(0.1*d_parsecs)) - extinct_mag
abs_mag_faint_G = mag_faint_G - 5*(log10(0.1*d_parsecs)) - extinct_mag

#Calculating colour index and correcting it of redenning effects for plotting on x-axis
colour_index_hot = mag_hot_BP-mag_hot_RP - excess_col
colour_index_faint = mag_faint_BP-mag_faint_RP - excess_col

Below, solar metallicity isochrones are read in. 

Note that they are taken from [here](http://waps.cfa.harvard.edu/MIST/model_grids.html#synthetic), from [Dotter 2016](http://adsabs.harvard.edu/abs/2016ApJS..222....8D) and [Choi et al., 2016](http://adsabs.harvard.edu/abs/2016ApJ...823..102C) from the MIST grid of stellar models. 

In [ ]:
isocmd = read_mist_models.ISOCMD('MIST_v1.2_feh_p0.00_afe_p0.0_vvcrit0.0_UBVRIplus.iso.cmd')

#Indices of four isochrone ages
AgeOne = isocmd.age_index(np.log10(0.5*10**9)) # 0.5Gyr
AgeTwo = isocmd.age_index(np.log10(1*10**9)) # 1Gyr
AgeThree = isocmd.age_index(np.log10(5*10**9)) # 5Gyr
AgeFour = isocmd.age_index(np.log10(10*10**9)) # 10Gyr

# Four isochrones
BP_one = isocmd.isocmds[AgeOne]['Gaia_BP_EDR3']
RP_one = isocmd.isocmds[AgeOne]['Gaia_RP_EDR3']
G_one = isocmd.isocmds[AgeOne]['Gaia_G_EDR3']

BP_two = isocmd.isocmds[AgeTwo]['Gaia_BP_EDR3']
RP_two = isocmd.isocmds[AgeTwo]['Gaia_RP_EDR3']
G_two = isocmd.isocmds[AgeTwo]['Gaia_G_EDR3']

BP_three = isocmd.isocmds[AgeThree]['Gaia_BP_EDR3']
RP_three = isocmd.isocmds[AgeThree]['Gaia_RP_EDR3']
G_three = isocmd.isocmds[AgeThree]['Gaia_G_EDR3']

BP_four = isocmd.isocmds[AgeFour]['Gaia_BP_EDR3']
RP_four = isocmd.isocmds[AgeFour]['Gaia_RP_EDR3']
G_four = isocmd.isocmds[AgeFour]['Gaia_G_EDR3']

Then Mamajak's table is read in for mass estimates

In [ ]:
mamjak_updated = pd.read_csv('EEM_dwarf_UBVIJHK_colors_Teff.txt',delim_whitespace=True,comment='#'
                      , names = ['SpT','BP-RP','M_G','Mass'],usecols=[0,11,13,30]
                      )[0:(141-23)].replace('...',np.NaN).replace('.....',np.NaN)
#print(mamjak_updated)
BP_RP_vals = mamjak_updated['BP-RP'].values.astype(float)
M_G_vals = mamjak_updated['M_G'].values.astype(float)
Mass_vals = mamjak_updated['Mass'].values.astype(float)

BP_RP_mass_hot = interp1d(BP_RP_vals,Mass_vals,bounds_error=False)(colour_index_hot.n)
BP_RP_mass_cool = interp1d(BP_RP_vals,Mass_vals,bounds_error=False)(colour_index_faint.n)

G_mass_hot = interp1d(M_G_vals,Mass_vals,bounds_error=False)(abs_mag_hot_G.n)
G_mass_cool = interp1d(M_G_vals,Mass_vals,bounds_error=False)(abs_mag_faint_G.n)

hot_mass_mean = np.mean([BP_RP_mass_hot,G_mass_hot])
cool_mass_mean = np.mean([BP_RP_mass_cool,G_mass_cool])

hot_mass_mean_u = np.std([BP_RP_mass_hot,G_mass_hot])/np.sqrt(2)
cool_mass_mean_u = np.std([BP_RP_mass_cool,G_mass_cool])/np.sqrt(2)

print('Mass estimate using BP-RP:')
print('Faint star: ',BP_RP_mass_cool,'Solar Masses')
print('Hot star: ',BP_RP_mass_hot,'Solar Masses')

print('Mass estimate using G:')
print('Faint star: ',G_mass_cool,'Solar Masses')
print('Hot star: ',G_mass_hot,'Solar Masses')

print('Average Estimates: ')
print('Hot star: ',hot_mass_mean,'+/-',hot_mass_mean_u,'Solar Masses')
print('Faint star: ',cool_mass_mean,'+/-',cool_mass_mean_u,'Solar Masses')


If the two estimates are similar (between BP-RP and G), the mass estimate can be used. If they aren't, disregard it.

In [ ]:
#Plotting the CMD
plt.figure()
plt.errorbar(colour_index_hot.n, abs_mag_hot_G.n, c='b', label='hotter star', xerr=colour_index_hot.s, yerr=abs_mag_hot_G.s, fmt='bx',capsize=3)
plt.errorbar(colour_index_faint.n, abs_mag_faint_G.n, c='r', label='fainter star', xerr=colour_index_faint.s, yerr=abs_mag_faint_G.s, fmt='rx',capsize=3)
plt.plot(BP_one-RP_one, G_one,c='aqua', label='0.5Gyr')
plt.plot(BP_two-RP_two, G_two,c='lightgreen', label='1Gyr')
plt.plot(BP_three-RP_three, G_three,c='gold', label='5Gyr')
plt.plot(BP_four-RP_four, G_four,c='salmon', label='10Gyr')
#plt.gca().invert_yaxis()
plt.ylim(10,-2)
plt.xlim(-0.25,2.5)
plt.xlabel('Colour Index, BP-RP [mag]')
plt.ylabel('Magnitude, G [mag]')
plt.title('A Colour-Magnitude Diagram of the stars in ' + str(binary_system))
plt.legend(loc='lower left')
plt.grid(True)

plt.show()